# Midterm Project Background Work
*Matthew Ueckermann*


## Importing the data

Can import one year as:

In [2]:
import pandas as pd

df_2019 = pd.read_csv('https://enviro.epa.gov/enviro/efservice/MV_TRI_BASIC_DOWNLOAD/year/=/2019/fname/TRI_2019_US.csv/CSV',low_memory=False)
# to scape later, do in for loop. Add time delay for politenes

print("done")

done


In [3]:
df_2019.head()

,1. YEAR,2. TRIFD,3. FRS ID,4. FACILITY NAME,5. STREET ADDRESS,6. CITY,7. COUNTY,8. ST,9. ZIP,10. BIA,...,108. 8.3 - ENERGY RECOVER OF,109. 8.4 - RECYCLING ON SITE,110. 8.5 - RECYCLING OFF SIT,111. 8.6 - TREATMENT ON SITE,112. 8.7 - TREATMENT OFF SITE,113. PRODUCTION WSTE (8.1-8.7),114. 8.8 - ONE-TIME RELEASE,115. PROD_RATIO_OR_ ACTIVITY,116. 8.9 - PRODUCTION RATIO,Unnamed: 116
0,2019,77015FRMNT2239H,110000460965,GB BIOSCIENCES LLC,2239 HADEN RD,HOUSTON,HARRIS,TX,77015,NaN,...,0.0,0.0,0.0,11.0,0.0,214.10,NaN,PRODUCTION,0.00,NaN
1,2019,2974WCMSTL6371C,110040511407,CM STEEL INC,6371 CAMPBELL ROAD,YORK,YORK,SC,29745,NaN,...,0.0,0.0,30200.0,0.0,0.0,30225.40,NaN,PRODUCTION,1.20,NaN
2,2019,77015FRMNT2239H,110000460965,GB BIOSCIENCES LLC,2239 HADEN RD,HOUSTON,HARRIS,TX,77015,NaN,...,0.0,0.0,0.0,0.0,0.0,13395.01,NaN,PRODUCTION,0.00,NaN
3,2019,5535WTCHTC1PREC,110031294882,TECH-ETCH INC,1 PRECISION DR,LITCHFIELD,MEEKER,MN,55355,NaN,...,0.0,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN
4,2019,51031WLLSD11911,110000414034,WELLS ENTERPRISES INC SOUTH ICE CREAM PLANT,1191 18TH ST SW,LE MARS,PLYMOUTH,IA,51031,NaN,...,0.0,0.0,0.0,156541.0,0.0,156541.00,NaN,PRODUCTION,0.97,NaN


In [18]:
# Want to select/keep only the data that we care about:
df_2019_trimmed = df_2019[["1. YEAR","2. TRIFD","8. ST","15. PARENT CO NAME","20. INDUSTRY SECTOR","34. CHEMICAL","38. CLEAN AIR ACT CHEMICAL","39. CLASSIFICATION","40. METAL","42. CARCINOGEN","45. 5.1 - FUGITIVE AIR","46. 5.2 - STACK AIR","47. 5.3 - WATER","51. 5.5.1 - LANDFILLS","59. ON-SITE RELEASE TOTAL","82. OFF-SITE RELEASE TOTAL","88. OFF-SITE RECYCLED TOTAL","101. TOTAL RELEASES","113. PRODUCTION WSTE (8.1-8.7)","114. 8.8 - ONE-TIME RELEASE","115. PROD_RATIO_OR_ ACTIVITY","116. 8.9 - PRODUCTION RATIO"]]

# this probably covers more than I'll actually use, but I think the rest of the columns are too much for me....

In [19]:
df_2019_trimmed.head()

,1. YEAR,2. TRIFD,8. ST,15. PARENT CO NAME,20. INDUSTRY SECTOR,34. CHEMICAL,38. CLEAN AIR ACT CHEMICAL,39. CLASSIFICATION,40. METAL,42. CARCINOGEN,...,47. 5.3 - WATER,51. 5.5.1 - LANDFILLS,59. ON-SITE RELEASE TOTAL,82. OFF-SITE RELEASE TOTAL,88. OFF-SITE RECYCLED TOTAL,101. TOTAL RELEASES,113. PRODUCTION WSTE (8.1-8.7),114. 8.8 - ONE-TIME RELEASE,115. PROD_RATIO_OR_ ACTIVITY,116. 8.9 - PRODUCTION RATIO
0,2019,77015FRMNT2239H,TX,SYNGENTA CORP,Chemicals,Hydrogen cyanide,NO,TRI,NO,NO,...,0.00,0,0.00,203.10,0.0,203.10,214.10,NaN,PRODUCTION,0.00
1,2019,2974WCMSTL6371C,SC,NaN,Fabricated Metals,Nickel,YES,TRI,YES,YES,...,0.00,0,25.40,0.00,30200.0,25.40,30225.40,NaN,PRODUCTION,1.20
2,2019,77015FRMNT2239H,TX,SYNGENTA CORP,Chemicals,Vanadium compounds,NO,TRI,YES,NO,...,262.57,0,264.57,13130.44,0.0,13395.01,13395.01,NaN,PRODUCTION,0.00
3,2019,5535WTCHTC1PREC,MN,TECH ETCH INC,Fabricated Metals,Chlorine,YES,TRI,NO,NO,...,0.00,0,0.00,0.00,0.0,0.00,0.00,NaN,NaN,NaN
4,2019,51031WLLSD11911,IA,WELLS ENTERPRISES INC,Food,Nitric acid,NO,TRI,NO,NO,...,0.00,0,0.00,0.00,0.0,0.00,156541.00,NaN,PRODUCTION,0.97
